# Core Imports and Setup

In [1]:
import os
import pandas as pd
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

import logging
# logging.getLogger("openff.toolkit").setLevel(logging.ERROR)
# logging.basicConfig(level=logging.DEBUG)

import json
from openff import toolkit, evaluator

In [2]:
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw, MolFromSmiles
from rdkit.Chem.Draw import MolsToGridImage, rdMolDraw2D, MolsMatrixToGridImage
import pubchempy
from PIL import Image, ImageDraw, ImageFont
IPythonConsole.ipython_useSVG=False  #< set this to False if you want PNGs instead of SVGs

In [3]:
from rdkit.Chem import rdMolDescriptors
import pubchempy as pcp
import math

In [4]:
from openff.evaluator.datasets import PhysicalProperty, PropertyPhase
from openff.evaluator.datasets.thermoml import thermoml_property
from openff.evaluator import properties
from openff.units import unit
from openff.evaluator.datasets.thermoml import ThermoMLDataSet
from openff.evaluator.datasets import PhysicalProperty, PropertyPhase, PhysicalPropertyDataSet

In [5]:
from openff.evaluator.datasets.curation.components.filtering import FilterByPropertyTypes, FilterByPropertyTypesSchema
from openff.evaluator.datasets.curation.components.filtering import FilterByTemperature, FilterByTemperatureSchema
from openff.evaluator.datasets.curation.components.filtering import FilterByPressure, FilterByPressureSchema
from openff.evaluator.datasets.curation.components.filtering import FilterBySmiles, FilterBySmilesSchema
from openff.evaluator.datasets.curation.components.filtering import FilterByNComponents, FilterByNComponentsSchema


In [6]:
from rich.progress import track
from collections import defaultdict
from openff.toolkit.utils.exceptions import UndefinedStereochemistryError

In [7]:
@thermoml_property("Osmotic coefficient", supported_phases=PropertyPhase.Liquid | PropertyPhase.Gas)
class OsmoticCoefficient(PhysicalProperty):
    """A class representation of a osmotic coeff property"""

    @classmethod
    def default_unit(cls):
        return unit.dimensionless
    
setattr(properties, OsmoticCoefficient.__name__, OsmoticCoefficient)

ThermoMLDataSet.registered_properties['Osmotic coefficient'].conversion_function

functools.partial(<function _default_mapping at 0x7f632b636cb0>, <class '__main__.OsmoticCoefficient'>)

In [8]:
def filter_database (unfiltered_directory):
    data_set=[]
    # Loop through all files in the directory
    for filename in os.listdir(unfiltered_directory):
        # Check if the file ends with .json
        if filename.endswith('.xml'):
            # Full path to the file
            file_path = os.path.join(unfiltered_directory, filename)
            data_set.append(file_path)
    
    sorted_dois = defaultdict(list)
    for doi in track(data_set, description='Filtering DOIs...'):
        try:
            dataset = ThermoMLDataSet._from_file(doi)
            sorted_dois['working'].append(doi)
        except UndefinedStereochemistryError:
            sorted_dois['stereo_fail'].append(doi)
        except Exception as other_exc:
            sorted_dois[other_exc.__class__.__name__].append(doi)

    print('Amount of failing files: %i/%i' % (len(sorted_dois['stereo_fail']),len(data_set)))
    print('Amount of working files: %i/%i' % (len(sorted_dois['working']),len(data_set)))

    return sorted_dois['working']

In [9]:
def extract_database (database_directory, csv_cached):
    CACHED_PROP_PATH = Path(csv_cached)

    if CACHED_PROP_PATH.exists():
        prop_df = pd.read_csv(CACHED_PROP_PATH, index_col=0)
        # delete rows with underfined thermo params to avoid pesky indexing errors
        prop_df = prop_df.dropna(subset=['Temperature (K)'])
        prop_df = prop_df.dropna(subset=['Pressure (kPa)'])
        data_set = ThermoMLDataSet.from_pandas(prop_df)
    else:
        sorted_files=filter_database(database_directory)
        data_set=ThermoMLDataSet.from_file(*sorted_files)

        initial_df = data_set.to_pandas()
        with CACHED_PROP_PATH.open('w') as file:
            initial_df.to_csv(CACHED_PROP_PATH)
    return data_set

In [10]:
# # Ensure database_directory is defined
database_directory = '10.1007'
# database_directory = 'tests'
tml_data_set = extract_database(database_directory, 'tml_database_07.csv')

Output()

Amount of failing files: 25/197
Amount of working files: 165/197


In [11]:
data_set=[]
# Loop through all files in the directory
for filename in os.listdir(database_directory):
    # Check if the file ends with .json
    if filename.endswith('.xml'):
        # Full path to the file
        file_path = os.path.join(database_directory, filename)
        data_set.append(file_path)

print(data_set)

['10.1007/s10765-008-0511-0.xml', '10.1007/s10765-010-0887-5.xml', '10.1007/s10765-006-0096-4.xml', '10.1007/s10765-015-1980-6.xml', '10.1007/s10765-011-0995-x.xml', '10.1007/s10765-009-0581-7.xml', '10.1007/s10765-019-2537-x.xml', '10.1007/s10765-010-0752-6.xml', '10.1007/s10765-008-0506-x.xml', '10.1007/s10765-012-1172-6.xml', '10.1007/s10765-008-0497-7.xml', '10.1007/s10765-016-2150-1.xml', '10.1007/s10765-013-1468-1.xml', '10.1007/s10765-016-2104-7.xml', '10.1007/s10765-009-0593-3.xml', '10.1007/s10765-011-1128-2.xml', '10.1007/s10765-008-0390-4.xml', '10.1007/s10765-009-0667-2.xml', '10.1007/s10765-013-1432-0.xml', '10.1007/s10765-011-1052-5.xml', '10.1007/s10765-008-0549-z.xml', '10.1007/s10765-005-8091-8.xml', '10.1007/s10765-008-0535-5.xml', '10.1007/s10765-015-2009-x.xml', '10.1007/s10765-016-2061-1.xml', '10.1007/s10765-006-0018-5.xml', '10.1007/s10765-014-1759-1.xml', '10.1007/s10765-012-1185-1.xml', '10.1007/s10765-009-0566-6.xml', '10.1007/s10765-005-5566-6.xml', '10.1007/

In [12]:
sorted_dois = defaultdict(list)
for doi in track(data_set, description='Filtering DOIs...'):
    try:
        dataset = ThermoMLDataSet._from_file(doi)
        sorted_dois['working'].append(doi)
    except UndefinedStereochemistryError:
        sorted_dois['stereo_fail'].append(doi)
    except Exception as other_exc:
        sorted_dois[other_exc.__class__.__name__].append(doi)

print('Amount of failing files: %i/%i' % (len(sorted_dois['stereo_fail']),len(data_set)))
print('Amount of working files: %i/%i' % (len(sorted_dois['working']),len(data_set)))

Output()

KeyboardInterrupt: 

In [ ]:
tml_data_set=ThermoMLDataSet.from_file(*sorted_dois['working'])

In [ ]:
tml_data_set

<PhysicalPropertyDataSet n_properties=537 n_substances=215 n_sources=2>

In [ ]:
tml_data_set.to_pandas()

,Id,Temperature (K),Pressure (kPa),Phase,N Components,Component 1,Role 1,Mole Fraction 1,Exact Amount 1,Component 2,Role 2,Mole Fraction 2,Exact Amount 2,Component 3,Role 3,Mole Fraction 3,Exact Amount 3,Density Value (g / ml),Density Uncertainty (g / ml),Source
0,c53292c345a2430b94b37c836fa44e1e,278.15,101.0,Liquid,3,O.O.O=S(=O)([O-])[O-].[Ca+2],Solvent,0.000360,None,[Cl-].[Na+],Solvent,0.001798,NaN,O,Solvent,0.997843,NaN,1.006000,0.001500,10.1021/acs.jced.5b00005
1,f1a8f964d7944e1d9466595e7f549816,278.15,101.0,Liquid,3,O.O.O=S(=O)([O-])[O-].[Ca+2],Solvent,0.000454,None,[Cl-].[Na+],Solvent,0.004499,NaN,O,Solvent,0.995047,NaN,1.014000,0.001500,10.1021/acs.jced.5b00005
2,b3cd061edd0f4d8b87a908cca61647c9,278.15,101.0,Liquid,3,O.O.O=S(=O)([O-])[O-].[Ca+2],Solvent,0.000591,None,[Cl-].[Na+],Solvent,0.008975,NaN,O,Solvent,0.990434,NaN,1.028000,0.001500,10.1021/acs.jced.5b00005
3,fb9b6f24eb914030855231a8d81b199b,278.15,101.0,Liquid,3,O.O.O=S(=O)([O-])[O-].[Ca+2],Solvent,0.000730,None,[Cl-].[Na+],Solvent,0.017961,NaN,O,Solvent,0.981308,NaN,1.048000,0.001000,10.1021/acs.jced.5b00005
4,abbc10af581f47a080f51323c6b51bb0,278.15,101.0,Liquid,3,O.O.O=S(=O)([O-])[O-].[Ca+2],Solvent,0.000809,None,[Cl-].[Na+],Solvent,0.027109,NaN,O,Solvent,0.972082,NaN,1.063000,0.001000,10.1021/acs.jced.5b00005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,b4a13d50731d482ba4484f14e0478793,323.15,101.0,Liquid,2,CCCC#N,Solvent,0.600000,None,c1ccccc1,Solvent,0.400000,NaN,NaN,NaN,NaN,NaN,0.798661,0.000063,10.1021/acs.jced.5b00009
533,454af844c84241c6af68f1a23b093fe6,323.15,101.0,Liquid,2,CCCC#N,Solvent,0.700000,None,c1ccccc1,Solvent,0.300000,NaN,NaN,NaN,NaN,NaN,0.789869,0.000065,10.1021/acs.jced.5b00009
534,5f26e10af9c04ee18ae1ac97813ae7bc,323.15,101.0,Liquid,2,CCCC#N,Solvent,0.800000,None,c1ccccc1,Solvent,0.200000,NaN,NaN,NaN,NaN,NaN,0.780977,0.000066,10.1021/acs.jced.5b00009
535,58c042e1e44d4d73aa95c21b58beeb19,323.15,101.0,Liquid,2,CCCC#N,Solvent,0.900000,None,c1ccccc1,Solvent,0.100000,NaN,NaN,NaN,NaN,NaN,0.771989,0.000068,10.1021/acs.jced.5b00009
